In [5]:
import pandas as pd

# Load the cleaned DWELDAT file
df = pd.read_csv('dweldat25_cleaned.csv', dtype=str)

# Ensure TMK13 is treated as a string and padded to 13 digits
df['TMK13'] = df['TMK13'].str.zfill(13)

# Extract TMK components
df['TMK_Island'] = df['TMK13'].str[0]
df['TMK_Zone'] = df['TMK13'].str[1:3]
df['TMK_Sec'] = df['TMK13'].str[3:5]
df['TMK_Plat'] = df['TMK13'].str[5:8]
df['TMK_Parcel'] = df['TMK13'].str[8:12]
df['TMK_CPR'] = df['TMK13'].str[12:]

# Compute 9-digit TMK (island + zone + section + plat + parcel)
df['TMK9'] = df['TMK13'].str[:12]

# Optional: reorder columns
cols = ['TMK13', 'TMK9', 'TMK_Island', 'TMK_Zone', 'TMK_Sec',
        'TMK_Plat', 'TMK_Parcel', 'TMK_CPR']
df_tmk = df[cols]

# Save to CSV
df_tmk.to_csv('tmk_parts.csv', index=False)

print("✅ TMK parts table created and saved as 'tmk_parts.csv'")


✅ TMK parts table created and saved as 'tmk_parts.csv'


In [7]:
import pandas as pd

# Load files
df_dweldat = pd.read_csv('dweldat25_cleaned.csv', dtype={'TMK13': str, 'Use_Code': str})
df_tmk_parts = pd.read_csv('tmk_parts.csv', dtype=str)

# Merge bedroom and TMK data
df = pd.merge(df_tmk_parts, df_dweldat[['TMK13', 'Bedrooms', 'Use_Code']], on='TMK13', how='left')

# Convert data types
df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
df['Use_Code'] = pd.to_numeric(df['Use_Code'], errors='coerce')

# Filter to Maui, with ≥1 bedroom and residential Use_Code
res_codes = list(range(100, 200))  # Residential use codes
res_df = df[
    (df['TMK_Island'] == '2') &
    (df['Bedrooms'] >= 1) &
    (df['Use_Code'].isin(res_codes))
].copy()

# Mark each 9-digit TMK with count of residential dwellings beneath it
tmk9_counts = res_df.groupby('TMK9').size().reset_index(name='Dwellings_On_Parcel')
res_df = pd.merge(res_df, tmk9_counts, on='TMK9', how='left')

# Add boolean column: True if this parcel has >1 residential dwelling (i.e. CPR/undercounted)
res_df['Potentially_Undercounted'] = res_df['Dwellings_On_Parcel'] > 1

# Save full detailed table (optional)
res_df[['TMK13', 'TMK9', 'Bedrooms', 'Dwellings_On_Parcel', 'Potentially_Undercounted']].to_csv('Maui_Residential_Dwellings.csv', index=False)

# Calculate summary stats
total_res_dwellings = res_df.shape[0]
undercounted = res_df[res_df['Potentially_Undercounted']].shape[0]
unique_tmks_with_cprs = res_df[res_df['Potentially_Undercounted']]['TMK9'].nunique()

# Print results
print(f"🏠 Total residential dwellings on Maui (TMK13 with ≥1 bedroom): {total_res_dwellings:,}")
print(f"⚠️ Undercounted dwellings sharing a TMK9 (likely CPR units): {undercounted:,}")
print(f"📍 Unique 9-digit TMKs containing ≥2 residential units: {unique_tmks_with_cprs:,}")


🏠 Total residential dwellings on Maui (TMK13 with ≥1 bedroom): 0
⚠️ Undercounted dwellings sharing a TMK9 (likely CPR units): 0
📍 Unique 9-digit TMKs containing ≥2 residential units: 0


In [9]:
# --- Fixed-Width Parser for DWELDAT File (Revised to Include Use_Code) ---
# Purpose: Read and label the full DWELDAT fixed-width text file
# Input: 'dweldat25.txt' (36MB fixed-width text)
# Output: Cleaned full table with headers + optional subset for TMK + BEDROOMS

import pandas as pd

# Step 1: Define fixed-width column specs (zero-based index, end is exclusive)
# Based on full DWELDAT layout from the official metadata PDF
colspecs = [
    (0, 13),    # TMK13
    (13, 17),   # Tax Year
    (17, 21),   # Sequence Number
    (21, 29),   # Land Area
    (29, 32),   # Number of Buildings
    (32, 37),   # Building Style Code
    (37, 42),   # Occupancy Code
    (42, 47),   # Roof Cover Code
    (47, 52),   # Roof Structure Code
    (52, 57),   # Ext Wall Code
    (57, 62),   # Int Wall Code
    (62, 67),   # Flooring Code
    (67, 71),   # Total Rooms
    (71, 76),   # Total Bathrooms
    (76, 81),   # Bedrooms
    (81, 86),   # Full Baths
    (86, 91),   # Half Baths
    (91, 96),   # Stories
    (96, 101),  # Units
    (101, 106), # Year Renovated
    (106, 111), # Effective Year Built
    (111, 116), # Condition Code
    (116, 121), # Quality Code
    (121, 126), # Use Code
    (126, 131), # Year Built
    (131, 136), # Living Area
    (136, 141), # Garage Area
    (141, 146), # Other Area
    (146, 151), # Carport Area
    (151, 156), # Lanai Area
    (156, 161), # Basement Area
    (161, 166), # Recreation Area
    (166, 171), # Other Improvements
    (171, 176), # Fireplace Count
    (176, 181), # Pool Code
    (181, 186), # Value Building
    (186, 191), # Value Land
    (191, 196), # Exemptions
    (196, 201), # Net Taxable Value
    (229, 239), # Total Assessed Value (Real Prop)
]

# Step 2: Define corresponding column names
column_names = [
    'TMK13', 'Tax_Year', 'Seq_No', 'Land_Area', 'Num_Bldgs', 'Bldg_Style_Code',
    'Occupancy_Code', 'Roof_Cover_Code', 'Roof_Structure_Code', 'Ext_Wall_Code',
    'Int_Wall_Code', 'Flooring_Code', 'Total_Rooms', 'Total_Bathrooms',
    'Bedrooms', 'Full_Baths', 'Half_Baths', 'Stories', 'Units',
    'Year_Renovated', 'Effective_Year_Built', 'Condition_Code', 'Quality_Code',
    'Use_Code', 'Year_Built', 'Living_Area', 'Garage_Area', 'Other_Area',
    'Carport_Area', 'Lanai_Area', 'Basement_Area', 'Recreation_Area',
    'Other_Improvements', 'Fireplace_Count', 'Pool_Code', 'Value_Building',
    'Value_Land', 'Exemptions', 'Net_Taxable_Value', 'Total_Assessed_Value'
]

# Step 3: Read the fixed-width file (adjust file path if needed)
file_path = 'dweldat25.txt'
df = pd.read_fwf(file_path, colspecs=colspecs, names=column_names, dtype=str)

# Step 4: Convert numeric fields
numeric_fields = [
    'Land_Area', 'Num_Bldgs', 'Total_Rooms', 'Total_Bathrooms', 'Bedrooms',
    'Full_Baths', 'Half_Baths', 'Stories', 'Units', 'Year_Renovated',
    'Effective_Year_Built', 'Year_Built', 'Living_Area', 'Garage_Area',
    'Other_Area', 'Carport_Area', 'Lanai_Area', 'Basement_Area',
    'Recreation_Area', 'Fireplace_Count', 'Value_Building', 'Value_Land',
    'Exemptions', 'Net_Taxable_Value', 'Total_Assessed_Value', 'Use_Code'
]

for col in numeric_fields:
    df[col] = pd.to_numeric(df[col].str.strip(), errors='coerce')

# Step 5: Save full cleaned table
df.to_csv('dweldat25_cleaned.csv', index=False)

# Step 6: Save TMK + Bedrooms table
df[['TMK13', 'Bedrooms']].to_csv('TMK_Bedrooms.csv', index=False)

print("✅ Parsing complete. Files saved:")
print(" - dweldat25_cleaned.csv")
print(" - TMK_Bedrooms.csv")


✅ Parsing complete. Files saved:
 - dweldat25_cleaned.csv
 - TMK_Bedrooms.csv


In [11]:
import pandas as pd

# Load your cleaned DWELDAT file
df = pd.read_csv("dweldat25_cleaned.csv", dtype=str)

# Convert 'Bedrooms' to numeric for filtering
df["Bedrooms"] = pd.to_numeric(df["Bedrooms"], errors="coerce")

# Step 1: Filter for Maui (Island code = '2') and dwellings with Bedrooms ≥ 1
df_maui = df[
    df["TMK13"].str.startswith("2") & 
    (df["Bedrooms"] >= 1)
].copy()

# Step 2: Extract 9-digit TMK root (first 9 characters)
df_maui["TMK9"] = df_maui["TMK13"].str[:9]

# Step 3: Count number of dwellings per 9-digit TMK
tmk_counts = df_maui.groupby("TMK9")["TMK13"].count().reset_index(name="Dwellings_Per_Parcel")

# Step 4: Identify parcels with multiple dwellings (potential CPRs)
shared_tmks = tmk_counts[tmk_counts["Dwellings_Per_Parcel"] > 1]

# Step 5: Filter full table for those CPR units
df_cpr_units = df_maui[df_maui["TMK9"].isin(shared_tmks["TMK9"])].copy()

# Step 6: Summary stats
total_dwellings = len(df_maui)
unique_9digit_tmks = df_maui["TMK9"].nunique()
potentially_undercounted_dwellings = len(df_cpr_units)
undercounted_tmks = shared_tmks["TMK9"].nunique()

# Display results
print("🏝️ Maui Residential Dwelling Analysis (Bedrooms ≥ 1)")
print(f"• Total residential dwellings: {total_dwellings:,}")
print(f"• Unique 9-digit parcels (TMK9): {unique_9digit_tmks:,}")
print(f"• Dwellings sharing 9-digit TMK (CPRs): {potentially_undercounted_dwellings:,}")
print(f"• Parcels affected by CPR undercounting: {undercounted_tmks:,}")

# Optional: Save filtered CPR units to file
df_cpr_units.to_csv("Maui_CPR_Dwellings.csv", index=False)


🏝️ Maui Residential Dwelling Analysis (Bedrooms ≥ 1)
• Total residential dwellings: 70,218
• Unique 9-digit parcels (TMK9): 37,085
• Dwellings sharing 9-digit TMK (CPRs): 41,718
• Parcels affected by CPR undercounting: 8,585
